In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np

In [2]:
# podemos dejar estas credenciasles y este archivo porque esto es una cuenta demo y sirve como guia educativa sobre pandas
mt5.initialize(
                path="C:\\Program Files\\Darwinex MetaTrader 5\\terminal64.exe",
                login=3000071302,
                password="@86P#icI2Q",
                server="demoUK-mt5.darwinex.com",
                timeout=60000,
                portable=False
            )

True

In [13]:
symbol = "EURUSD"
tf = mt5.TIMEFRAME_H1
from_position = 1 # 0 significa vela actual 1 seria la ultima vela cerrada
num_bars = 15 # cantidad de velas apartir de from_position

bars = pd.DataFrame(mt5.copy_rates_from_pos(symbol, tf, from_position, num_bars))
bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars.set_index('time', inplace=True)
bars.rename(columns={'tick_volume': 'tickvol', 'real_volume': 'vol'}, inplace=True)
bars = bars[['open', 'high', 'low', 'close', 'tickvol', 'vol', 'spread']]

bars.head()

,open,high,low,close,tickvol,vol,spread
time,,,,,,,
2024-10-14 16:00:00,1.09094,1.09273,1.09047,1.09190,4185,0,0
2024-10-14 17:00:00,1.09190,1.09190,1.09050,1.09148,3934,0,0
2024-10-14 18:00:00,1.09148,1.09165,1.09048,1.09070,2221,0,0
2024-10-14 19:00:00,1.09070,1.09086,1.09007,1.09015,1368,0,0
2024-10-14 20:00:00,1.09015,1.09042,1.08974,1.09040,987,0,0


In [15]:
bars['close'].rolling(10).mean().iloc[-1] # calcular la media movil de los ultimos 10 precios de cierre
bars['close'][-10:].mean() # calcular la media de los ultimos 10 precios de cierre
np.mean(bars['close'][-10:]) # calcular la media de los ultimos 10 precios de cierre

1.090472

In [8]:
# Una serie es un objeto de pandas que representa una columna de un DataFrame
# Una serie es un array unidimensional con etiquetas
cierre_vela = bars['close']
cierre_vela

time
2024-10-10 15:00:00    1.09440
2024-10-10 16:00:00    1.09372
2024-10-10 17:00:00    1.09222
2024-10-10 18:00:00    1.09248
2024-10-10 19:00:00    1.09059
Name: close, dtype: float64

In [9]:
# recuperar el ultimo item de la serie
bars.iloc[-1]

open          1.09248
high          1.09281
low           1.09031
close         1.09059
tickvol    4068.00000
vol           0.00000
spread        0.00000
Name: 2024-10-10 19:00:00, dtype: float64

In [14]:
latest_bar = bars.iloc[-1]
latest_bar.name

Timestamp('2024-10-10 19:00:00')

In [11]:
last_tick = mt5.symbol_info_tick('EURUSD')._asdict()
last_tick

{'time': 1728594404,
 'bid': 1.09211,
 'ask': 1.09213,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1728594404360,
 'flags': 6,
 'volume_real': 0.0}

In [12]:
def get_latest_tick(symbol: str) -> dict:
    """
    Retrieves the latest tick for the given symbol.

    Parameters:
    symbol (str): The symbol for which to retrieve the latest tick.

    Returns:
    dict: A dictionary containing the latest tick information.
    """
    try:
        tick = mt5.symbol_info_tick(symbol)
        if tick is None:
            print(f"No se ha podido recuperar el último tick de {symbol} - MT5 error: {mt5.last_error()}")
            return {}
    
    except Exception as e:
        print(f"Algo no ha ido bien a la hora de recuperar el último tick de {symbol}. MT5 error: {mt5.last_error()}, exception: {e}")
    
    else:
        return tick._asdict() # _asdict() es un metodo de mt5 para convertir el objeto en un diccionario de python

In [13]:
get_latest_tick("EURUSD")

{'time': 1728594966,
 'bid': 1.0923,
 'ask': 1.09233,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1728594966691,
 'flags': 4,
 'volume_real': 0.0}